# Imports

In [1]:
from keras import backend as K
import time
import matplotlib.pyplot as plt
import numpy as np_utils
%matplotlib inline

from keras.models import Sequential
from keras.layers.convolutional import Convolution2D, MaxPooling2D, DepthwiseConv2D, Conv2D, SeparableConv2D, MaxPooling1D
from keras.layers import Input, concatenate
import gensim.downloader as api
from sklearn.model_selection import train_test_split
from keras.layers import Activation, Flatten, Dense, Dropout
from keras.layers.normalization import BatchNormalization
from keras.layers.pooling import GlobalAveragePooling2D
from keras.utils import np_utils
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import SGD, Nadam, Adam
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau, LearningRateScheduler
from keras.regularizers import l2
%config InlineBackend.figure_format='retina'
from keras_contrib.callbacks import CyclicLR
from keras.models import Model
from keras.layers import Conv1D, GlobalMaxPooling1D, GlobalAveragePooling1D
from data_science_utils.vision.keras import *
from time import time
import pandas as pd
import numpy as np

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
import missingno as msno
import re
from joblib import Parallel, delayed
from data_science_utils import dataframe as df_utils
from data_science_utils import models as model_utils
from data_science_utils import plots as plot_utils
from data_science_utils.dataframe import column as column_utils
from data_science_utils import misc as misc
from data_science_utils import preprocessing as pp_utils
from data_science_utils import nlp as nlp_utils

from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
from keras.layers import Embedding
from keras.layers import Conv1D, GlobalMaxPooling1D
from keras.datasets import imdb
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from data_science_utils.dataframe import get_specific_cols

import numpy as np
import more_itertools
from more_itertools import flatten
import ast
from sklearn.preprocessing import LabelEncoder

Using TensorFlow backend.


[nltk_data] Downloading package punkt to /home/ec2-user/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /home/ec2-user/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /home/ec2-user/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/ec2-user/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package omw to /home/ec2-user/nltk_data...
[nltk_data]   Package omw is already up-to-date!


In [2]:
df_train = pd.read_csv("price_prediction/training.csv")
df_test = pd.read_csv("price_prediction/public_test_features.csv")

# Preprocessing Data

In [18]:
import sys
import os
sys.path.append(os.getcwd())
from importlib import reload
import lib
reload(lib)
from lib import *

<module 'lib' from '/home/ec2-user/SageMaker/ML_hackathon_2019/lib.py'>

In [19]:
df_train = preprocess_for_fasttext_cmd(df_train,jobs=32)
df_test = preprocess_for_fasttext_cmd(df_test,jobs=32)


In [51]:
df_train['RPRICE'] = np.ceil(np.sqrt(df_train['PRICE']))

# df_train = df_train[df_train['RPRICE']<=100]

df_train['label'] = '__label__'

df_train['label'] = df_train['label'] + df_train['RPRICE'].astype(str)

# Proposing to cutoff at 70

In [17]:
df_train[df_train['RPRICE']>=80]

,ID,ASIN_ID,TITLE,BULLET_POINTS,GL,PRICE,char,RPRICE,label
159465,527271,B00OE1GHOO,CaratLane 18K 3 Color Gold and Diamond Bangle,[This product can be returned in 10 days. Exch...,gl_jewelry,5.520500e+05,caratlane 18k 3 color gold and diamond bangle[...,83.0,__label__83.0
289847,614509,B074QVZD3H,"Apple iPhone 8 Plus (Space Grey, 64GB) Pack of 10",NaN,gl_wireless,6.021964e+05,"apple iphone 8 plus (space grey, 64gb) pack of...",85.0,__label__85.0
358827,971197,B01DPPPCM2,PNY Quadro M6000 24 GB PCI-E Graphics Card,"[30-Bit color (10-bit per each red, green and ...",gl_pc,5.826263e+05,pny quadro m6000 24 gb pci-e graphics card[30-...,84.0,__label__84.0
452742,1699146,B016870I8W,Parker New Made In France Duofold Giant 125Th ...,NaN,gl_office_product,1.700000e+06,parker new made in france duofold giant 125th ...,120.0,__label__120.0
488180,421016,B00JXPN9NI,HR Wireless Rubberized Cover for HTC One Remix...,"[Practical & Stylish,Allows access to all butt...",gl_wireless,6.818202e+07,hr wireless rubberized cover for htc one remix...,409.0,__label__409.0
502065,18145,B005GV2YIY,Frederique Constant Women's FC-303WHD2PD6B Lad...,NaN,gl_watch,6.736380e+05,frederique constant women's fc-303whd2pd6b lad...,88.0,__label__88.0
699407,490251,B06Y31N26X,Thermo Nano Drop Lite UV Visible Spectrophotom...,NaN,gl_biss,4.978000e+05,thermo nano drop lite uv visible spectrophotom...,80.0,__label__80.0
705598,1538873,B01F1ONULU,P.N.Gadgil Jewellers Splashing Colored 18k Whi...,"[This ring is made from 18k gold,This ring has...",gl_jewelry,1.614873e+06,p.n.gadgil jewellers splashing colored 18k whi...,118.0,__label__118.0
843241,892470,B01LTIBO1U,"Apple iPhone 7 Plus (Rose Gold, 128GB) - Pack ...","[12MP primary cameras with optical zoom at 2x,...",gl_wireless,5.644196e+05,"apple iphone 7 plus (rose gold, 128gb) - pack ...",83.0,__label__83.0
866783,959256,B0773DJ6CB,S. K. Industries Paper Bag Making Machine,"[Easy To Use,High Quality & Durable,Perfect Fi...",gl_biss,5.084746e+05,s. k. industries paper bag making machine[easy...,80.0,__label__80.0


In [52]:
df_train['label'].nunique()
pd.DataFrame(df_train['label'].value_counts()).head(70).tail(20)

499

,label
__label__62.0,2630
__label__64.0,2345
__label__61.0,2106
__label__66.0,2057
__label__63.0,2038
__label__67.0,1877
__label__69.0,1738
__label__65.0,1736
__label__68.0,1617
__label__71.0,1595


In [53]:
df_train['text'] = df_train['label']
df_train['text'] = df_train['text'] + " "
df_train['text'] = df_train['text'] + df_train['char']

In [54]:
df_test['text'] = df_test['char']

In [24]:
df_train.head()

,ID,ASIN_ID,TITLE,BULLET_POINTS,GL,PRICE,char,RPRICE,label,text
0,1489960,B07HNQBLZH,"Livsmart Resin Jewellery Display Stand, 17x11c...",[Holds rings and other small items especially ...,gl_jewelry,507.62,livsmart resin jewellery display stand 17x11c...,8.0,__label__8.0,__label__8.0 livsmart resin jewellery display ...
1,633493,B00MTZ726Y,Quantum Creation: Does the Supernatural Lurk i...,NaN,gl_digital_ebook_purchase,479.90,quantum creation does the supernatural lurk i...,8.0,__label__8.0,__label__8.0 quantum creation does the supern...
2,1474591,B015MZ3NJW,IZOD Men's Casual Shirt (8907163477392_ZKSH019...,"[Cotton linen,Cotton linen shirt,Checkered pat...",gl_apparel,829.28,izod men s casual shirt 8907163477392_zksh019...,10.0,__label__10.0,__label__10.0 izod men s casual shirt 8907163...
3,830218,B079DNYB4N,"Dishaa Women's Rayon A-Line Kurta (Black, X-La...","[Material: Rayon,Color: Black ; Work: Threaded...",gl_apparel,648.31,dishaa women s rayon a line kurta black x la...,9.0,__label__9.0,__label__9.0 dishaa women s rayon a line kurta...
4,201144,B00XHH7LBY,Return of the Ravens (Ulfrik Ormsson's Saga Bo...,NaN,gl_digital_ebook_purchase,332.00,return of the ravens ulfrik ormsson s saga bo...,7.0,__label__7.0,__label__7.0 return of the ravens ulfrik orms...


In [55]:
train,test = train_test_split(df_train[['text']],test_size=0.25)

train[['text']].to_csv("fastText-0.2.0/train-sqrt-1.txt",header=False,index=False)
test[['text']].to_csv("fastText-0.2.0/train-sqrt-2.txt",header=False,index=False)

In [56]:
df_train[['text']].to_csv("fastText-0.2.0/train-sqrt.txt",header=False,index=False)

!head -n5 fastText-0.2.0/train.txt

__label__8.0 livsmart resin jewellery display stand  17x11cm  black[holds rings and other small items especially non hanging items each hand is specially designed to hang ring  bracelet or necklace beautiful gesture jewelry organizer ideal for retail shop counter display and personal use  helps you to organize rings]gl_jewelry
__label__8.0 quantum creation  does the supernatural lurk in the fourth dimension  gl_digital_ebook_purchase
__label__10.0 izod men s casual shirt  8907163477392_zksh0191_x large_navy [cotton linen cotton linen shirt checkered pattern with plain weave machine wash  do not bleach and wring  medium heat iron made in india]gl_apparel
__label__9.0 dishaa women s rayon a line kurta  black  x large    readymade kurti   embroidered kurti[material  rayon color  black   work  threaded embroidered fit type  slim fit  wash care  first time dry wash only neck style  banded collar   bust size  42   waist size  37   hip size  44 length  knee long  type of sleeves  3 4 sleeve]g

In [57]:
df_test[['text']].to_csv("fastText-0.2.0/test-sqrt.txt",header=False,index=False)

```
./fasttext supervised -input train-1.txt -output price_prediction -lr 0.1 -epoch 10

./fasttext test price_prediction.bin train-2.txt

./fasttext predict price_prediction.bin test.txt > prediction.txt
```

# Now Using Predictions from CMD line

In [58]:
df_results = pd.read_csv("fastText-0.2.0/prediction-sqrt.txt",header=None)

df_results.shape
df_results.columns=["result"]
df_results.head()

(362403, 1)

,result
0,__label__19.0
1,__label__17.0
2,__label__17.0
3,__label__17.0
4,__label__30.0


In [59]:
df_results['values'] = df_results['result'].apply(lambda x:x[9:]).astype(float)
df_results['values'] = np.power(df_results['values'],2)
df_results.head()

,result,values
0,__label__19.0,361.0
1,__label__17.0,289.0
2,__label__17.0,289.0
3,__label__17.0,289.0
4,__label__30.0,900.0


In [60]:
df_sub = df_test[['ID']]
df_sub['PRICE'] = df_results['values']
df_sub.to_csv("fasttext-square.csv",index=False)

/home/ec2-user/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app


In [61]:
np.sum(pd.isna(df_sub))

ID       0
PRICE    0
dtype: int64

In [46]:
!head -n10 fasttext-cubic.csv

ID,PRICE
1585751,343.0
1530678,512.0
1324955,343.0
822454,1331.0
1604015,512.0
1141644,512.0
1209134,3375.0
935730,729.0
158576,343.0


https://fasttext.cc/docs/en/supervised-tutorial.html